In [1]:
import tensorflow as tf
import data_utils
import my_translate as translate
import seq2seq_model

In [2]:
from_train, to_train, from_dev, to_dev = translate.prepare_data()

In [3]:
print(from_train, to_train, from_dev, to_dev)

data/train/namuwiki_chosung.txt.ids10000 data/train/namuwiki_sentences.txt.ids10000 data/dev/namuwiki_chosung_dev.txt.ids10000 data/dev/namuwiki_sentences_dev.txt.ids10000


In [4]:
#padding을 해야함
import numpy as np

chosung_list = open(from_dev,'r')
sentence_list = open(to_dev,'r')
chosungs = []
sentences = []

line_counter = 0
while True:    
    chosung_line = chosung_list.readline()
    sentence_line = sentence_list.readline()
    if not chosung_line: break
    chosungs.append(np.array([int(index) for index in chosung_line.split()]))
    sentences.append(np.array([data_utils.GO_ID] + [int(index) for index in sentence_line.split()]))
    line_counter += 1
chosung_list.close()
sentence_list.close()

print(line_counter)
print(len(chosungs))
print(len(sentences))

10
10
10


In [5]:
weights = np.ones(10, dtype=np.float32)

#     for length_idx in xrange(decoder_size):
#       batch_decoder_inputs.append(
#           np.array([decoder_inputs[batch_idx][length_idx]
#                     for batch_idx in xrange(self.batch_size)], dtype=np.int32))

#       # Create target_weights to be 0 for targets that are padding.
#       batch_weight = np.ones(self.batch_size, dtype=np.float32)
#       for batch_idx in xrange(self.batch_size):
#         # We set weight to 0 if the corresponding target is a PAD symbol.
#         # The corresponding target is decoder_input shifted by 1 forward.
#         if length_idx < decoder_size - 1:
#           target = decoder_inputs[batch_idx][length_idx + 1]
#         if length_idx == decoder_size - 1 or target == data_utils.PAD_ID:
#           batch_weight[batch_idx] = 0.0
#       batch_weights.append(batch_weight)

In [6]:
dtype=tf.float32
encoder_inputs = []
decoder_inputs = []
target_weights = []
for i in range(10):
    encoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                            name="encoder{0}".format(i)))
    decoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                            name="decoder{0}".format(i)))
    target_weights.append(tf.placeholder(dtype, shape=[None],
                                                name="weight{0}".format(i)))
    
# print(encoder_inputs)
# print(decoder_inputs)
# print(target_weights)

In [7]:
input_feed = {}
for i in range(10):
    input_feed[encoder_inputs[i].name] = chosungs[i]
    input_feed[decoder_inputs[i].name] = sentences[i]
#     input_feed[target_weights[i].name] = weights[i]

In [8]:
size = 64
cell = tf.contrib.rnn.GRUCell(size)

In [9]:
# with tf.variable_scope("seq2seq2"):
outputs, state = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(
          encoder_inputs,
          decoder_inputs,
          cell,
          num_encoder_symbols=1000,
          num_decoder_symbols=1000,
          embedding_size=size,
          dtype=dtype)

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    test_outputs, test_state = sess.run([outputs, state], feed_dict=input_feed)

InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [37,64] vs. shape[1] = [23,64]
	 [[Node: embedding_attention_seq2seq/rnn/gru_cell_2/gates/gates/concat = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_attention_seq2seq/rnn/embedding_wrapper_2/embedding_lookup, embedding_attention_seq2seq/rnn/gru_cell_1/add, embedding_attention_seq2seq/rnn/gru_cell_2/gates/gates/concat/axis)]]

Caused by op 'embedding_attention_seq2seq/rnn/gru_cell_2/gates/gates/concat', defined at:
  File "/home/tf_user/anaconda3/envs/tf/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/tf_user/anaconda3/envs/tf/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/tf_user/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/tf_user/anaconda3/envs/tf/lib/python3.5/site-package